In [2]:
import pandas as pd
import numpy as np
import random
from rdkit import Chem
from snn_model import SNNet, device, train_snn, test_snn, get_loss_fn, num_hidden
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from snntorch import spikegen, surrogate
import matplotlib.pyplot as plt
from utils import load_dataset_df, fp_generator
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score
from sklearn.utils.class_weight import compute_class_weight

import os
import tempfile
from ray import train, tune
from ray.tune.schedulers import ASHAScheduler
from ray.train import Checkpoint, report

#### Load DataFrame

In [5]:
files = ['tox21.csv','sider.csv', 'BBBP.csv']
dt_file = files[2]

df, targets = load_dataset_df(filename=dt_file)
print(targets)

target_name = targets[0]
df = df[[target_name, 'smiles']].dropna()
print(df.size)
df.drop_duplicates()
print(df.size)

['p_np']
4100
4100


#### SMILE to Fingerprint

In [3]:
fp_types = [['morgan', 1024], ['maccs', 167], ['RDKit', 1024]]
fp_type, num_bits = fp_types[1]
print(fp_type, '-', num_bits)
num_rows = len(df)
fp_array = np.zeros((num_rows, num_bits))
target_array = np.zeros((num_rows, 1))
i = 0

img = None
# Smile to Fingerprint of size {num_bits}
fp_gen = fp_generator(fp_type)
for idx, row in df.iterrows():
    mol = Chem.MolFromSmiles(row['smiles'])
    #TODO: sanitize molecules to remove the warnings (?)
    
    if mol is not None:
        fingerprint = fp_gen(mol)

        fp_array[i] = np.array(fingerprint)
        target_array[i] = row[target_name]
        i += 1
target_array = target_array.ravel()

maccs - 167


[19:14:49] Explicit valence for atom # 1 N, 4, is greater than permitted
[19:14:49] WARNING: not removing hydrogen atom without neighbors
[19:14:49] Explicit valence for atom # 6 N, 4, is greater than permitted
[19:14:49] WARNING: not removing hydrogen atom without neighbors
[19:14:49] WARNING: not removing hydrogen atom without neighbors
[19:14:49] WARNING: not removing hydrogen atom without neighbors
[19:14:49] WARNING: not removing hydrogen atom without neighbors
[19:14:49] WARNING: not removing hydrogen atom without neighbors
[19:14:49] WARNING: not removing hydrogen atom without neighbors
[19:14:50] Explicit valence for atom # 6 N, 4, is greater than permitted
[19:14:50] WARNING: not removing hydrogen atom without neighbors
[19:14:50] WARNING: not removing hydrogen atom without neighbors
[19:14:50] WARNING: not removing hydrogen atom without neighbors
[19:14:50] WARNING: not removing hydrogen atom without neighbors
[19:14:50] Explicit valence for atom # 11 N, 4, is greater than pe

In [4]:
# Create Torch Dataset
dtype = torch.float32
fp_tensor = torch.tensor(fp_array, dtype=dtype)
target_tensor = torch.tensor(target_array, dtype=dtype).long()

dataset = TensorDataset(fp_tensor, target_tensor)

#### Train/Test Split

In [ ]:
generator = torch.Generator().manual_seed(1)
train, val, test = random_split(dataset, [0.8, 0.1, 0.1], generator=generator) #experimentar 0.9, 0.1

_, train_label = train[:]
_, val_label = val[:]
_, test_label = test[:]


#### Loss Function

In [ ]:
loss_types = ['cross_entropy', 'rate_loss', 'temporal_loss']
loss_type = loss_types[1]
print(loss_type)

use_weights = True

#Loss Function
if use_weights:
    class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=np.array(train_label))
    class_weights = torch.tensor(class_weights, dtype=torch.float)
    class_weights[0] += 1
    weighted = 'class_weights'
else: 
    class_weights=None
    weighted = ''

loss_fn = get_loss_fn(loss_type=loss_type, class_weights=class_weights)

cross_entropy


#### Ray Tune

In [9]:
random.seed(1)

In [ ]:
def tune_train(config,  checkpoint_dir=None):
    # Extract the hyperparameters to be used by the model
    net = Net(num_inputs=num_bits,num_steps=config["time_steps"], spike_grad=None, use_l2=True).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"], betas=(0.9, 0.999), weight_decay=0)

    train_loader = DataLoader(train, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(val, batch_size=config["batch_size"], shuffle=True)
    test_loader = DataLoader(test, batch_size=config["batch_size"], shuffle=True)


    for i in range(config["num_iterations"]):
        net, _ = train_snn(
                net=net,
                optimizer=optimizer,
                device=device,
                num_epochs=config["num_epochs"],
                train_loader=train_loader,
                val_loader=val_loader,
                loss_type=config["loss_type"],
                loss_fn=config["loss_fn"],
                dtype=config["dtype"]
        )
        all_preds, all_targets = test_snn(net, device, test_loader)
        auc_roc = roc_auc_score(all_targets, all_preds)

        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            checkpoint = None
            if (i + 1) % 5 == 0:
                # This saves the model to the trial directory
                torch.save(
                    net.state_dict(),
                    os.path.join(temp_checkpoint_dir, "model.pth")
                )
                checkpoint = Checkpoint.from_directory(temp_checkpoint_dir)

            # Send the current training result back to Tune
            #suggestion: auc * acc
            report({"mean_auc": auc_roc}, checkpoint=checkpoint)


In [ ]:
config = {
    "loss_type": loss_type,
    "loss_fn": loss_fn,
    "lr": tune.grid_search([1e-6, 1e-5, 1e-4, 1e-3]),
    #"batch_size": tune.grid_search([16, 32, 64]),
    "batch_size": tune.grid_search([32, 64]),
    "num_epochs": 1000, # mudar para 1000 com early stopping
    "dtype": torch.float32,
    "num_iterations": 1,
    "time_steps": tune.grid_search([5, 10, 15, 20])
}


In [ ]:
tuner = tune.Tuner(
    tune_train,
    tune_config=tune.TuneConfig(
        num_samples=50,
        scheduler=ASHAScheduler(metric="mean_auc", mode="max"),
    ),
    param_space=config,

)
results = tuner.fit()

c:\Users\knsve\Desktop\MEI\Tese\torch\pt_venv\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
(pid=264